<a href="https://colab.research.google.com/github/danielkorat/allennlp-dm-parse-probs/blob/master/Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Select backend and pre-trained model

In [1]:
FRAMEWORK = 'tf' # or 'pt'
MODEL_NAME = 'distilbert-base-uncased'
SMOKE_TEST = True

In [13]:
from sys import executable
! {executable} -m pip install -q -q transformers==4.4.2

In [3]:
from pathlib import Path
from sklearn.model_selection import train_test_split

import transformers
transformers.logging.set_verbosity_info()

# Download Dataset

In [4]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar -xf aclImdb_v1.tar.gz

--2021-03-22 16:50:34--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  25.0MB/s    in 3.6s    

2021-03-22 16:50:38 (22.4 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



# Read Dataset

In [5]:
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)
    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

if SMOKE_TEST:
    train_texts, train_labels = train_texts[:500], train_labels[:500]
    test_texts, test_labels = test_texts[:200], test_labels[:200]

# Preprocess

## Tokenize

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

https://huggingface.co/distilbert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7hxts8oh


storing https://huggingface.co/distilbert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
creating metadata file for /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_

storing https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1dhhpbwl


storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpg_8djtxp


storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4


## Convert to tensors

In [7]:
if FRAMEWORK == 'tf':
    import tensorflow as tf
    from tensorflow.data import Dataset
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    from transformers import TFAutoModelForSequenceClassification as model_cls
    from transformers import TFTrainer as trainer_cls
    from transformers import TFTrainingArguments as training_args_cls

    train_dataset = Dataset.from_tensor_slices((dict(train_encodings), train_labels))
    test_dataset = Dataset.from_tensor_slices((dict(test_encodings), test_labels))

if FRAMEWORK == 'pt':
    import torch

    from transformers import AutoModelForSequenceClassification as model_cls
    from transformers import Trainer as trainer_cls
    from transformers import TrainingArguments as training_args_cls

    class IMDbDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

    train_dataset = IMDbDataset(train_encodings, train_labels)
    test_dataset = IMDbDataset(test_encodings, test_labels)

# Define training arguments

In [8]:
training_args = training_args_cls(
    output_dir='./results',             # output directory
    num_train_epochs=5,                 # total number of training epochs
    max_steps=20 if SMOKE_TEST else -1, # total number of training steps (overrides `num_train_epochs`)
    per_device_train_batch_size=16,     # batch size per device during training
    warmup_steps=500,                   # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                  # strength of weight decay
    logging_dir='./logs',               # directory for storing logs
    logging_steps=10,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


# Load pre-trained model

In [9]:
if FRAMEWORK == 'tf':
    with training_args.strategy.scope():
        model = model_cls.from_pretrained(MODEL_NAME)
if FRAMEWORK == 'pt':
    model = model_cls.from_pretrained(MODEL_NAME)

Tensorflow: setting up strategy
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.4.2",
  "vocab_size": 30522
}

https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpc

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
creating metadata file for /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [10]:
trainer = trainer_cls(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

You are instantiating a Trainer but W&B is not installed. To use wandb logging, run `pip install wandb; wandb login` see https://docs.wandb.com/huggingface.
To use comet_ml logging, run `pip/conda install comet_ml` see https://www.comet.ml/docs/python-sdk/huggingface/
***** Running training *****
  Num examples = 500
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 32
  Total optimization steps = 50
{'loss': 0.73637736, 'learning_rate': 1e-06, 'epoch': 0.3125, 'step': 10}
{'loss': 0.7247439, 'learning_rate': 2e-06, 'epoch': 0.625, 'step': 20}
{'loss': 0.7045751, 'learning_rate': 2.9999999e-06, 'epoch': 0.9375, 'step': 30}
{'loss': 0.5781995, 'learning_rate': 4e-06, 'epoch': 1.25, 'step': 40}
{'loss': 0.5152919, 'learning_rate': 5.0000003e-06, 'epoch': 1.5625, 'step': 50}
Training took: 0:00:55.321360


# Fine-Tune

In [ ]:
trainer.train()

# Inference

In [11]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples in dataset = 200
  Batch size = 8


PredictionOutput(predictions=array([[-0.48878717,  0.27526283],
       [-0.40666398,  0.32540652],
       [-0.39781886,  0.23536034],
       [-0.5084412 ,  0.34741974],
       [-0.5841867 ,  0.32355973],
       [-0.54209816,  0.32265845],
       [-0.4760672 ,  0.27736217],
       [-0.35989612,  0.17264113],
       [-0.5286444 ,  0.2894849 ],
       [-0.5097582 ,  0.31521928],
       [-0.5541358 ,  0.33417195],
       [-0.4479991 ,  0.35061246],
       [-0.421418  ,  0.27657336],
       [-0.6145901 ,  0.33127484],
       [-0.5634431 ,  0.3728723 ],
       [-0.5468086 ,  0.32598224],
       [-0.5918125 ,  0.34967583],
       [-0.49347717,  0.3319161 ],
       [-0.46901673,  0.27988037],
       [-0.4921293 ,  0.2682007 ],
       [-0.5248504 ,  0.3463697 ],
       [-0.4142467 ,  0.20636764],
       [-0.5804467 ,  0.26634616],
       [-0.55745333,  0.3780276 ],
       [-0.4787989 ,  0.32782227],
       [-0.4407545 ,  0.20198174],
       [-0.48187667,  0.32064924],
       [-0.4748394 ,  0.34